# College Admissions and the stability of marriage by Gale and Shapley

In [60]:
import numpy as np
import matplotlib.pyplot as plt

## College Admission

### Notations
#### n - number of applicants
#### m - number of colleges
#### q - quota or seats in each college i
### Rules
#### 1. each applicants rank the college in order to his/her preference and omitting only those college he would never accept on any circumstances.
#### 2. each college rank the students in order to it preferences and omitting those students which they will never admit theme on any circumstances. even it means not filling its quota or seats.
### QUES: What problem this algorithm solves?
#### ANS: An assignment of the applicants to colleges will be called unstable if there is two applicants x and y who are assigned to colleges A and B respectively although x prefer B over A and B prefer x over y.  
### Theorem 1
#### There always exists a stable set of marriages.
### Proof - 

In [61]:
applicants = [
    {
        'name': "u",
        'preference': ['A', 'B', 'C'],
        'blacklist': [],
        'rejected': []
    },
    {
        'name': "v",
        'preference': ['B', 'A', 'C'],
        'blacklist': [],
        'rejected': []
    },
    {
        'name': "w",
        'preference': ['C', 'B', 'A'],
        'blacklist': [],
        'rejected': []
    },
    {
        'name': "x",
        'preference': ['A', 'B'],
        'blacklist': ['C'],
        'rejected': []
    },
    {
        'name': "y",
        'preference': ['A', 'B'],
        'blacklist': ['C'],
        'rejected': []
    },
    {
        'name': "z",
        'preference': ['C', 'A', 'B'],
        'blacklist': [],
        'rejected': []
    },
    
]
colleges = [
    {
        'name': "A",
        'quota': 2,
        'preference': ['u', 'x', 'y', 'v', 'w'],
        'not_eligible': ['z'],
        'waiting_list': []
    },
    {
        'name': "B",
        'quota': 2,
        'preference': ['x', 'z', 'w', 'y', 'v'],
        'not_eligible': ['u'],
        'waiting_list': []
    },
    {
        'name': "C",
        'quota': 2,
        'preference': ['y', 'x', 'u', 'w', 'v'],
        'not_eligible': ['z'],
        'waiting_list': []
    },
]

#### 1. Each applicants send job application to first prefer college.
#### 2. Each college who received more than one applicant proposal will reject all except her favorite from those proposal.
#### 3. college will not finalised that applicant but wait for someone better.
#### 4. when a applicant send proposal to college then result is optimal for applicant and when college send proposal to applicant it is optimal for college.

In [62]:
def get_colleges_name_list(colleges):
    """
    return name list of all colleges
    :param colleges: list of college
    :return: list
    """
    return list(map(lambda x : x['name'], colleges))

def get_applicants_name_list(applicants):
    """
    return name list of all colleges
    :param colleges: list of college
    :return: list
    """
    return list(map(lambda x : x['name'], applicants))
    
colleges_name_list = get_colleges_name_list(colleges)
applicants_name_list = get_applicants_name_list(applicants)

def is_equal_lists(list1, list2):
    """
    check two python list
    :param list1: list 1
    :param list2: list 2
    :return: True if two list are equal
    """
    list1.sort()
    list2.sort()
    
    return list1 == list2

def get_total_waiting_list(colleges):
    """
    return all applicants which are in waiting list of any college
    :param colleges: colleges list
    :return: Boolean
    """
    
    waiting_list = [j for i in map(lambda x : x['waiting_list'], colleges) for j in i]
    
    return waiting_list

def is_match_complete(applicants, colleges):
    """
    The procedure return true when every applicant is either on waiting list q of any college or
    has been rejected by every college to which he is willing and permitted to apply.
    :param applicants: applicants list
    :param colleges: colleges list
    :return: Boolean
    """
    global colleges_name_list
    global applicants_name_list
    waiting_rejected_list = []
    
    # get all students who are rejected by all colleges
    waiting_rejected_list.extend(map(lambda x : x['name'], list(filter(lambda x : is_equal_lists(x['rejected'], colleges_name_list) , applicants))))
    
    # get all students who are in waiting list of any college
    waiting_rejected_list.extend(get_total_waiting_list(colleges))
    
    # check if any student left who are not rejected by all college or not in waiting list
    if not is_equal_lists(applicants_name_list, waiting_rejected_list):
        return False
    
    return True

def get_applicant_proposal(applicant, colleges):
    """
    get proposal for applicant
    :param applicant: 
    :return: Dict
    """
    for item in applicant['preference']:
        if not item in applicant['rejected'] and not item in get_total_waiting_list(colleges):
            return (applicant['name'], item)

def get_proposals(remaining_applicants, colleges):
    """
    find proposal of all remaining applicants to colleges
    :param remaining_applicants: 
    :return: return a dict {'<college name>': <applicant name> }
    """
    
    # find next favorite college for remaining applicants
    proposals = {i:j for i,j in map(lambda x : get_applicant_proposal(x, colleges), remaining_applicants)}
    
    return proposals

def get_lowest_prefer_appicant(waiting_list, preference):
    """
    find lowest prefer applicant in waiting list
    :param waiting_list: 
    :param preference: 
    :return: return applicant name
    """
    preference_index = list(map(lambda x: preference.index(x), waiting_list))
    preference_index.sort()
    
    return preference[preference_index[-1]]

def new_waiting_list(college, applicant_name):
    """
    get new waiting list if college accept this new candidate
    :param college: college data
    :param applicant_name: name of applicant
    :return: 
    """
    # if applicant is not in preference than college will reject it
    if not applicant_name in college['preference']:
        return college['waiting_list'], False
    
    # if size of waiting list is less than quota we will add that applicant
    if college['quota'] > len(college['waiting_list']):
        return college['waiting_list'].append(applicant_name), True
    
    # if applicant have higher preference than applicant that are in waiting list then replace it with lowest preference
    lowest_prefer_applicant = get_lowest_prefer_appicant(college['waiting_list'], college['preference'])
    if college['preference'].index(lowest_prefer_applicant) > college['preference'].index(applicant_name):
        college['waiting_list'].remove(lowest_prefer_applicant)
        college['waiting_list'].append(applicant_name)
        return college['waiting_list'], True
    
    return college['waiting_list'], False
        

def update_rejected_list(applicants, applicant_name, college_name):
    """
    update rejected list of applicant name
    :param applicants: 
    :param applicant_name: 
    :param college_name: 
    :return: 
    """
    for applicant in applicants:
        if applicant_name == applicant['name']:
            applicant['rejected'].append(college_name)
    
    return applicants

def update_waiting_list(colleges, proposals, applicants):
    """
    update each college list based on proposal received and college preference
    :param colleges: college data
    :param proposals: proposal of applicant
    :return: 
    """
    for applicant_name, college_name in zip(proposals.keys(), proposals.values()):
        for college in colleges:
            if college['name'] == college_name:
                _ , is_selected = new_waiting_list(college, applicant_name)
                if not is_selected:
                    applicants = update_rejected_list(applicants, applicant_name, college['name'])
    
    return colleges, applicants

count = 0
colleges_history = []
while not is_match_complete(applicants, colleges):
    # get total waiting list
    total_waiting_list = get_total_waiting_list(colleges)
    
    # applicant that is not in total waiting list will send proposal to next favorite college
    remaining_applicants_name = list(set(applicants_name_list) - set(total_waiting_list))
    remaining_applicants = list(filter(lambda x: x['name'] in remaining_applicants_name, applicants))
    proposals = get_proposals(remaining_applicants, colleges)
    
    # college will receive proposal from applicants and will decide to accept or reject
    # update waiting list of each college
    colleges, applicants = update_waiting_list(colleges, proposals, applicants)

    count += 1
    print('iteration '+str(count)+' finish')

iteration 1 finish
iteration 2 finish
iteration 3 finish
iteration 4 finish
iteration 5 finish
iteration 6 finish


In [63]:
colleges



[{'name': 'A',
  'quota': 2,
  'preference': ['u', 'x', 'y', 'v', 'w'],
  'not_eligible': ['z'],
  'waiting_list': ['u', 'x']},
 {'name': 'B',
  'quota': 2,
  'preference': ['x', 'z', 'w', 'y', 'v'],
  'not_eligible': ['u'],
  'waiting_list': ['y', 'z']},
 {'name': 'C',
  'quota': 2,
  'preference': ['y', 'x', 'u', 'w', 'v'],
  'not_eligible': ['z'],
  'waiting_list': ['w', 'v']}]